# Geth Docker Setup
For Geth, we will have to run this script first then run docker compose file later.

## Todo

- [ ] Know why this works lol!!!

In [ ]:
import os

path = os.path.abspath("")
print(path)

store_path = os.path.join(path, "geth/")
print(store_path)

env_path = os.path.abspath(os.path.join(path, ".."))
print(env_path)

In [ ]:
# Get "GETH_NETWORK_ID" and "GETH_NETWORK_NAME" from .env file
import dotenv

env_file = os.path.join(env_path, ".env")
print(env_file)

dotenv.load_dotenv(env_file)

network_id = os.getenv("GETH_NETWORK_ID")
network_acc = os.getenv("GETH_ACCOUNT_PASSWORD")

if network_id is None or network_acc is None:
    raise Exception("GETH_NETWORK_ID or GETH_NETWORK_NAME is not set")

print(network_id)
print(network_acc)

In [ ]:
# Copy to .env inside store_path (create if not exists)
import shutil

store_env_file = os.path.join(store_path, ".env")
print(store_env_file)

shutil.copy(env_file, store_env_file)